In [15]:
import os

import imageio
import torch

In [10]:
IMG = '../../../../img'

In [11]:
img_arr = imageio.imread(f'{IMG}/bobby.jpg')
img_arr.shape

(720, 1280, 3)

In [13]:
img = torch.from_numpy(img_arr)
out = img.permute(2, 0, 1) # HxWxC -> CxHxW (torch format)
out.shape

torch.Size([3, 720, 1280])

In [14]:
batch_size = 3
# batch x C x H x W
batch = torch.zeros(batch_size, 3, 256, 256, dtype=torch.uint8)

In [16]:
CAT_DIR = f'{IMG}/cats'

In [21]:
filenames = [f for f in os.listdir(CAT_DIR) if f.endswith('.png')]
filenames

['cat1.png', 'cat2.png', 'cat3.png']

In [22]:
for i, f in enumerate(filenames):
    img_arr = imageio.imread(os.path.join(CAT_DIR, f))
    img_t = torch.from_numpy(img_arr)
    img_t = img_t.permute(2, 0, 1)
    img_t = img_t[:3] # drop alpha if present
    batch[i] = img_t